In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
from collections import defaultdict

# Try to pull all Friday releases for movies in April 2016
url = 'http://www.imdb.com/movies-coming-soon/2016-04'
response = requests.get(url)
while(not response.ok):
    response = requests.get(url)

page = response.text
soup = BeautifulSoup(page,'lxml')

# Pulling upcoming movies from IMDB
release_dict = defaultdict(list)
for item in soup.find_all('h4'):
    item = item.find('a').text
    if(item[0] != " "):
        value = item.strip()
    else:
        name = item.split('(')[0].strip()
        release_dict[name].append(value)
        release_dict[name].append(item.split('(')[1].replace(')',''))
        
release_dict = pd.DataFrame.from_dict(release_dict,orient='index')
release_dict = release_dict.reset_index()
release_dict.columns = ['Title','Date','Year']
release_dict

,Title,Date,Year
0,Nina,April 22,2016
1,Mr. Right,April 8,2015
2,The Boss,April 8,2016
3,Everybody Wants Some!!,April 1,2016
4,The Meddler,April 22,2015
5,A Hologram for the King,April 22,2016
6,Tale of Tales,April 22,2015
7,The Invitation,April 8,2015
8,A Space Program,April 8,2015
9,Colonia,April 15,2015


In [3]:
# Pull opening weekend revenue for 2015
url = 'http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm'
response = requests.get(url)
while(not response.ok):
    respone = requests.get(url)
    
page = response.text
soup = BeautifulSoup(page,'lxml')

In [7]:
def scrape_opening_weekend(year=2015):
    url = 'http://www.boxofficemojo.com/yearly/chart/?yr='+str(year)+'&p=.htm'
    response = requests.get(url)
    while(not response.ok):
        response = requests.get(url)

    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    # Find number of pages on table for the year
    item = soup.find('center')
    pages = len(item.find_all('a'))+1
    
    # Pull table from each page
    for i in range(1,pages+1):
        url = 'http://www.boxofficemojo.com/yearly/chart/?page='+str(i)+'&view=releasedate&view2=domestic&yr='+str(year)+'&sort=opengross&p=.htm'
        tables = pd.read_html(url)
        table = tables[2]
        table = table.iloc[6:-6,2:9]
        table.columns = ['title','studio','total_gross','total_theatres','opening','opening_theatres','release_date']
        if i==1:
            df = table
        else:
            df = pd.concat([df,table])
    # Clean up dataframe types
    df.release_date = pd.to_datetime
    return df
        

In [8]:
data = scrape_opening_weekend()


In [16]:
data

,title,studio,total_gross,total_theatres,opening,opening_theatres,release_date
6,Star Wars: The Force Awakens,BV,"$935,832,841",4134,"$247,966,675",4134,<function to_datetime at 0x109d84d90>
7,Jurassic World,Uni.,"$652,270,625",4291,"$208,806,270",4274,<function to_datetime at 0x109d84d90>
8,Avengers: Age of Ultron,BV,"$459,005,868",4276,"$191,271,109",4276,<function to_datetime at 0x109d84d90>
9,Furious 7,Uni.,"$353,007,020",4022,"$147,187,040",4004,<function to_datetime at 0x109d84d90>
10,Minions,Uni.,"$336,045,770",4311,"$115,718,405",4301,<function to_datetime at 0x109d84d90>
11,The Hunger Games: Mockingjay - Part 2,LGF,"$281,723,902",4175,"$102,665,981",4175,<function to_datetime at 0x109d84d90>
12,Inside Out,BV,"$356,461,711",4158,"$90,440,272",3946,<function to_datetime at 0x109d84d90>
13,Fifty Shades of Grey,Uni.,"$166,167,230",3655,"$85,171,450",3646,<function to_datetime at 0x109d84d90>
14,Spectre,Sony,"$200,074,609",3929,"$70,403,148",3929,<function to_datetime at 0x109d84d90>
15,Pitch Perfect 2,Uni.,"$184,296,230",3660,"$69,216,890",3473,<function to_datetime at 0x109d84d90>


In [9]:
movies_currentyear = scrape_opening_weekend(2016)

In [10]:
movies_currentyear.head()

,title,studio,total_gross,total_theatres,opening,opening_theatres,release_date
6,Batman v Superman: Dawn of Justice,WB,"$313,381,486",4256,"$166,007,347",4242,<function to_datetime at 0x109d84d90>
7,Deadpool,Fox,"$360,361,721",3856,"$132,434,639",3558,<function to_datetime at 0x109d84d90>
8,The Jungle Book (2016),BV,"$125,094,414",4028,"$103,261,464",4028,<function to_datetime at 0x109d84d90>
9,Zootopia,BV,"$309,258,949",3959,"$75,063,401",3827,<function to_datetime at 0x109d84d90>
10,Kung Fu Panda 3,Fox,"$141,363,717",3987,"$41,282,042",3955,<function to_datetime at 0x109d84d90>


In [11]:
#helping rafa
url = 'http://www.boxofficemojo.com/search/?q=x-men:%20the%20last%20stand'
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')

In [12]:
soup.find_all('b')[1].find('a').get('href')

'/movies/?id=x3.htm'

In [13]:
soup.find('tr',attrs={'bgcolor':'#FFFF99'}).find('a').get('href')

'/movies/?id=x3.htm'

In [14]:
BeautifulSoup.find_all?

In [15]:
import numpy as np
np.NaN

nan

In [193]:
for year in range(1982,2017):
    url = 'http://www.boxofficemojo.com/monthly/?view=calendargross&chart=byyear&yr='+str(year)+'&view=calendargross'
    tables = pd.read_html(url,header=0)
    table = tables[2]
    table = table.iloc[:-1,:]
    table['year'] = year
    if year == 1982:
        df = table
    else:
        df = pd.concat([df, table])

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [179]:
url = 'http://www.boxofficemojo.com/monthly/?view=calendargross&chart=byyear&yr=2015&view=calendargross'
tables = pd.read_html(url,header=0)

In [191]:
df.head()

,Month,Total Gross,% ofYear,Movies,Avg.,#1 Movie,Gross,% ofTotal
0,DECEMBER,$245.8,7.8%,30,$8.2,Tootsie,$32.1,13.1%
1,NOVEMBER,$159.9,5.1%,29,$5.5,First Blood,$22.4,14.0%
2,OCTOBER,$144.6,4.6%,30,$4.8,E.T.,$21.2,14.7%
3,SEPTEMBER,$194.5,6.1%,28,$6.9,E.T.,$28.0,14.4%
4,AUGUST,$357.7,11.3%,29,$12.3,E.T.,$63.0,17.6%


In [194]:
df

,Month,Total Gross,% ofYear,Movies,Avg.,#1 Movie,Gross,% ofTotal,year
0,DECEMBER,$245.8,7.8%,30,$8.2,Tootsie,$32.1,13.1%,1982
1,NOVEMBER,$159.9,5.1%,29,$5.5,First Blood,$22.4,14.0%,1982
2,OCTOBER,$144.6,4.6%,30,$4.8,E.T.,$21.2,14.7%,1982
3,SEPTEMBER,$194.5,6.1%,28,$6.9,E.T.,$28.0,14.4%,1982
4,AUGUST,$357.7,11.3%,29,$12.3,E.T.,$63.0,17.6%,1982
5,JULY,$398.4,12.6%,29,$13.7,E.T.,$99.3,24.9%,1982
6,JUNE,$381.3,12.1%,29,$13.1,E.T.,$67.0,17.6%,1982
7,MAY,$170.9,5.4%,26,$6.6,Conan the Barbarian,$27.8,16.3%,1982
8,APRIL,$196.8,6.2%,23,$8.6,Porky's,$41.7,21.2%,1982
9,MARCH,$201.1,6.4%,28,$7.2,On Golden Pond,$31.8,15.8%,1982


In [186]:
table = tables[2]
table = table.iloc[:-1,:]

,Month,Total Gross,% ofYear,Movies,Avg.,#1 Movie,Gross,% ofTotal
0,DECEMBER,"$1,305.0",11.7%,176,$7.4,Star Wars: The Force Awakens,$652.0,50.0%
1,NOVEMBER,$891.3,8.0%,204,$4.4,The Hunger Games: Mockingjay - Part 2,$201.1,22.6%
2,OCTOBER,$715.2,6.4%,230,$3.1,The Martian,$179.4,25.1%
3,SEPTEMBER,$626.4,5.6%,205,$3.1,Hotel Transylvania 2,$55.8,8.9%
4,AUGUST,$844.1,7.6%,180,$4.7,M:I - Rogue Nation,$150.7,17.8%
5,JULY,"$1,183.6",10.6%,183,$6.5,Minions,$278.8,23.6%
6,JUNE,"$1,233.7",11.1%,172,$7.2,Jurassic World,$514.4,41.7%
7,MAY,"$1,074.5",9.7%,188,$5.7,Avengers: Age of Ultron,$427.6,39.8%
8,APRIL,$778.3,7.0%,182,$4.3,Furious 7,$324.4,41.7%
9,MARCH,$704.5,6.3%,177,$4.0,Cinderella,$153.2,21.8%


[Empty DataFrame
 Columns: [Adjuster:  Actuals  Est. Tckts  for(i=2016; i>=1933; i--) {  document.write('<option value="' + i + '"'); 	if(i=='0') document.write(' selected'); 	document.write('>' + i ); 	if(i=='0') document.write(', $' + '0.00'); 	document.write('</option>');  }  1929  1924  1910]
 Index: [], Empty DataFrame
 Columns: [ALL RELEASES, WIDE RELEASES, LIMITED RELEASES, CALENDAR GROSSES]
 Index: []]